# faceRec

This IPython notebook will classify the emotion of a face

Cohn-Kanade AU-Coded Expression Database - CK+
 
123 subjects (85 female)

emotional expressions were scored/validated, so there are a variable number of emotions / subject 

happy, sad, surprise, neutral, anger, disgust, fear 


CK+ Notes:

The Emotion coded files (Emotion_labels.zip) - ONLY 327 of the 593 sequences have emotion sequences. This is because these are the only ones the fit the prototypic definition. Like the FACS files, there is only 1 Emotion file for each sequence which is the last frame (the peak frame). There should be only one entry and the number will range from 0-7 (i.e. 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise). N.B there is only 327 files- IF THERE IS NO FILE IT MEANS THAT THERE IS NO EMOTION LABEL (sorry to be explicit but this will avoid confusion).

# Find the faces and emotion files

In [70]:
import glob
import os
import numpy as np
import cv2
from time import time
import cPickle as pickle
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Save file
saveDir = '/Users/abock/Desktop/InsightProject/websiteFiles/'
saveFile = '/Users/abock/Desktop/InsightProject/websiteFiles/save.p'
saveMovie = '/Users/abock/Desktop/InsightProject/websiteFiles/faceMovie.mp4'

# Initialize variables
subjectNames = []
subjectFiles = []
subjectEmotions = []
subjectImages = []

emotions = ['neutral','anger','contempt','disgust','fear','happy','sadness','surprise']
emotionLabels = ['anger','disgust','happy','neutral','sadness','surprise']

# OpenCV face classifier
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

# Face processing defaults
outImgSize = 500
scaleSize = 1.05
numNeighbors = 3
faceSize = 150

# Classifier
n_components = 50

### CK+ images

In [6]:
# Set paths, variable values
dbDir = '/Users/abock/Desktop/InsightProject/faceDatabases/CK+/'
emoDir = '/Users/abock/Desktop/InsightProject/faceDatabases/CK+/Emotion/'
# emoNums = [1,3,4,5,0,6,7]
emoNums = [3,1,6,0,5,7]

print("Loading input images...")
t0 = time()

# Find the images and emotion labels
ct = 0;
thisEmotion = []
for subDir in glob.glob(emoDir + 'S*/'):
    subject = os.path.basename(os.path.dirname(subDir))
    # Append to subjectNames
    subjectNames.append(subject)
    for faceDir in glob.glob(subDir + '*/'):
        dirNum = os.path.basename(os.path.dirname(faceDir))
        for thisFile in glob.glob(faceDir + '*.txt'):
            imDir = os.path.join(dbDir,subject,dirNum)
            fName = os.path.basename(thisFile[:-12])
            imFile = os.path.join(imDir,fName + '.png')
            
            with open(thisFile) as f:
                for line in f:
                    line = map(float, line.split())
                    lineInt = int(line[0])
#                     if lineInt != 2: # 2 is contempt
                    if lineInt != 2 and lineInt !=4: # 2 is contempt, 4 is fear
                        thisEmotion = emotions[lineInt]

            # Load the emotion image
            rawImg = cv2.imread(imFile)

            # Detect faces in the image
            faces = faceCascade.detectMultiScale(
            rawImg,
            scaleFactor=scaleSize,
            minNeighbors=numNeighbors,
            minSize=(faceSize, faceSize),
            flags = cv2.CASCADE_SCALE_IMAGE
            )

            # If a face is detected
            if len(faces):
                # Convert to gray
                gImg = cv2.cvtColor(rawImg, cv2.COLOR_BGR2GRAY)

                # Resize just the face
                x,y,w,h = faces[0,:]
                faceImg = gImg[y:y+h,x:x+w]
                img = cv2.resize(faceImg,(outImgSize,outImgSize),interpolation = cv2.INTER_CUBIC)

#                 cv2.imshow('Image',img)
#                 cv2.waitKey(30)

                # Flatten
                l = img.shape[0] * img.shape[1]
                img = img.reshape(1, l)
                
                # Append to subjectNames
                subjectNames.append(subject)
                # Append to subjectEmotions
                subjectEmotions.append(thisEmotion)
                # Append to subjectFiles
                subjectFiles.append(imFile) 
                # Append to output matrices
                subjectImages.append(img[0]) 
                
            # Load the neutral image
            imFile = os.path.join(imDir,fName[:-3] + '001.png')
            rawImg = cv2.imread(imFile)

            # Detect faces in the image
            faces = faceCascade.detectMultiScale(
            rawImg,
            scaleFactor=scaleSize,
            minNeighbors=numNeighbors,
            minSize=(faceSize, faceSize),
            flags = cv2.CASCADE_SCALE_IMAGE
            )

            # If a face is detected
            if len(faces):
                # Convert to gray
                gImg = cv2.cvtColor(rawImg, cv2.COLOR_BGR2GRAY)

                # Resize just the face
                x,y,w,h = faces[0,:]
                faceImg = gImg[y:y+h,x:x+w]
                img = cv2.resize(faceImg,(outImgSize,outImgSize),interpolation = cv2.INTER_CUBIC)

#                 cv2.imshow('Image',img)
#                 cv2.waitKey(30)

                # Flatten
                l = img.shape[0] * img.shape[1]
                img = img.reshape(1, l)
                
                # Append to subjectNames
                subjectNames.append(subject)
                # Append to subjectEmotions
                subjectEmotions.append('neutral')
                # Append to subjectFiles
                subjectFiles.append(imFile) 
                # Append to output matrices
                subjectImages.append(img[0]) 
                
# Convert to numpy array    
subjectNames = np.asarray(subjectNames)
subjectEmotions = np.asarray(subjectEmotions)
subjectFiles = np.asarray(subjectFiles)
subjectImages = np.asarray(subjectImages)

print("done in %0.3fs" % (time() - t0))

Loading input images...
done in 39.538s


### Train a model

In [7]:
X_train = subjectImages
Y_train = subjectEmotions

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
print("done in %0.3fs" % (time() - t0))

print("Fitting the classifier to the training set")
t0 = time()

param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
#clf = GridSearchCV(SVC(kernel='rbf',probability=True), param_grid)

clf = clf.fit(X_train_pca, Y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Extracting the top 50 eigenfaces from 654 faces
done in 15.473s
Projecting the input data on the eigenfaces orthonormal basis
done in 1.349s
Fitting the classifier to the training set
done in 11.847s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [8]:
for emotion in emotions:
    print(emotion)
    emotionSummary = np.array(subjectEmotions == emotion)
    print(np.sum(emotionSummary))

neutral
327
anger
56
contempt
0
disgust
62
fear
0
happy
73
sadness
45
surprise
91


In [9]:
# Save out the pca and classifier
print("Saving the training set to file")
t0 = time()
pickle.dump([pca,clf],open(saveFile,'w'))
print("done in %0.3fs" % (time() - t0))

Saving the training set to file
done in 7.086s


# Train / test

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(
    subjectImages, subjectEmotions, test_size=0.2, random_state=42)

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
print("done in %0.3fs" % (time() - t0))

print("Fitting the classifier to the training set")
t0 = time()

param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

#clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = GridSearchCV(SVC(kernel='rbf',probability=True), param_grid)

# metrics = ['minkowski','euclidean','manhattan'] 
# weights = ['uniform','distance'] #10.0**np.arange(-5,4)
# NN = np.arange(1,30)
# #algorithms = ['ball_tree', 'kd_tree', 'brute']
# algorithms = ['brute']
# param_grid = dict(metric=metrics,weights=weights,n_neighbors=NN,algorithm=algorithms)
# clf = GridSearchCV(neighbors.KNeighborsClassifier(), param_grid)

clf = clf.fit(X_train_pca, Y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Extracting the top 50 eigenfaces from 523 faces
done in 14.433s
Projecting the input data on the eigenfaces orthonormal basis
done in 1.371s
Fitting the classifier to the training set
done in 8.872s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [11]:
X_test_pca = pca.transform(X_test)
Y_pred = clf.predict(X_test_pca)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

      anger       0.55      0.30      0.39        20
    disgust       0.67      0.75      0.71         8
      happy       0.93      0.81      0.87        16
    neutral       0.79      0.89      0.84        65
    sadness       0.40      0.57      0.47         7
   surprise       0.79      0.73      0.76        15

avg / total       0.74      0.75      0.74       131



# Novel image

In [13]:
print("Loading the training set to file")
t0 = time()
pca, clf = pickle.load(open(saveFile,'rb'))
print("done in %0.3fs" % (time() - t0))

Loading the training set to file
done in 32.117s


In [1]:
#inFace = '/Users/abock/Desktop/InsightProject/sad_keanu.png'
#inFace = '/Users/abock/Desktop/InsightProject/crazy-guy.png'
inFace = '/Users/abock/Desktop/InsightProject/happy_guy.png'
#inFace = '/Users/abock/Desktop/InsightProject/happy_girl.png'
#inFace = '/Users/abock/Desktop/InsightProject/smiling-women.jpg'
#inFace = '/Users/abock/Desktop/InsightProject/sadGoslingMeme.jpg'
#inFace = '/Users/abock/Desktop/InsightProject/sadDawson.jpg'

#inFace = '/Users/abock/Desktop/InsightProject/aia_sadness_547.png'
#inFace = '/Users/abock/Desktop/InsightProject/aia_anger_93.png'
#inFace = '/Users/abock/Desktop/InsightProject/malcolm_joy_104.png'

cv2.destroyAllWindows()

outImgSize = 500
scaleSize = 1.05
numNeighbors = 3
faceSize = 10

# Load the image
rawImg = cv2.imread(inFace)
img = rawImg
img = cv2.resize(rawImg, (outImgSize, outImgSize), interpolation=cv2.INTER_CUBIC)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    img,
    scaleFactor=scaleSize,
    minNeighbors=1,
    minSize=(faceSize, faceSize),
    flags=cv2.CASCADE_SCALE_IMAGE
)

# Get the largest face
i = np.where(faces[:, 2] == faces[:, 2].max())
faces = faces[i, :]
faces = faces[0, :]
x, y, w, h = faces[0, :]

# Rectangle on face
faceImage = img
cv2.rectangle(faceImage, (x, y), (x + w, y + h), (0, 255, 0), 4)
cv2.imshow('Face',faceImage)
cv2.waitKey(30)

# Resize just the face
croppedImage = img[y:y + h, x:x + w]

# Classify input image
if len(faces):
    # Convert to gray
    gImg = cv2.cvtColor(croppedImage, cv2.COLOR_BGR2GRAY)

    # Resize just the face
    img = cv2.resize(gImg, (outImgSize, outImgSize), interpolation=cv2.INTER_CUBIC)

    # Flatten
    l = img.shape[0] * img.shape[1]
    img = img.reshape(1, l)

    testFace = np.asarray(img)
    X_test_pca = pca.transform(testFace)
    Y_pred = clf.predict(X_test_pca)
    print(Y_pred)

    # Classification figure
    probs = clf.predict_proba(X_test_pca)
    df = DataFrame(probs, columns=emotionLabels)
    ax = df.plot.bar()
    ax.xaxis.set_visible(False)
    ax.set_ylim([0, 1])
    ax.set_ylabel('Probability')
    plt.show()

NameError: name 'cv2' is not defined

In [61]:
cv2.destroyAllWindows()

# Record Movie

In [66]:
import numpy as np
import cv2
import time
import os

outDir = '/Users/abock/Desktop/faceMovie/'
if not os.path.isdir(outDir):
    os.makedirs(outDir)

cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
cap = cv2.VideoCapture(0)
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH);
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT); 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(saveMovie,fourcc, 15.0, (int(w),int(h)))

time.sleep(5) # wait for camera to catch up
ct = 0
while ct < 100:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if ret==True:
        #frame = cv2.flip(frame,0)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        cv2.waitKey(30)

    ct = ct + 1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

### Save movie as frames

In [72]:
cap = cv2.VideoCapture(saveMovie)
success,image = cap.read()
count = 0
success = True
while success:
  success,image = cap.read()
  cv2.imwrite(saveDir + "frame%d.jpg" % count, image)     # save frame as JPEG file
  count += 1
    
cap.release()

# Download images

In [ ]:
from bs4 import BeautifulSoup
import urllib2 
import urllib
import os

def saveURLimages(url,outDir):
    req = urllib2.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib2.urlopen(req).read()
    soup = BeautifulSoup(html)
    # get images
    images = [img for img in soup.findAll('img')]
    print (str(len(images)) + " images found.")
    print 'Downloading images to: ' + outDir
    
    # download images
    image_links = [each.get('src') for each in images]
    #print(image_links)
    for each in image_links:
        if not each is None:
            if each[0:4] == 'http':
                #print(each)
                filename=each.split('/')[-1]
                urllib.urlretrieve(each,os.path.join(outDir,filename + '.jpg'))

### Google

In [ ]:
# Save happy faces (Google Images)
happyPerson = 'https://www.google.com/search?site=&tbm=isch&source=hp&biw=1680&bih=949&q=happy+person&oq=happy+person&gs_l=img.3..0l10.15311.17067.0.17213.12.7.0.5.5.0.82.450.7.7.0....0...1ac.1.64.img..0.12.461.FpA8-IvoCOk'
url = happyPerson
outDir = '/Users/abock/Desktop/HappyFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

# Save sad faces (Google Images)
sadPerson = 'https://www.google.com/search?site=&tbm=isch&source=hp&biw=840&bih=949&q=sad+person&oq=sad+person&gs_l=img.3..0l10.732.2268.0.2413.10.7.0.3.3.0.337.1010.0j1j2j1.4.0....0...1ac.1.64.img..3.7.1015.rkrdqlI1p-k'
url = sadPerson
outDir = '/Users/abock/Desktop/SadFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

### Bing

In [ ]:
# Save happy faces (Bing Images)
happyPerson = 'https://www.bing.com/images/search?q=happy%20person&qs=n&form=QBIR&pq=happy%20person&sc=8-12&sp=-1&sk='
url = happyPerson
outDir = '/Users/abock/Desktop/HappyFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

# Save sad faces (Bing Images)
sadPerson = 'https://www.bing.com/images/search?q=sad+person&qs=n&form=QBILPG&pq=sad+person&sc=8-8&sp=-1&sk='
url = sadPerson
outDir = '/Users/abock/Desktop/SadFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

### Happy Faces

In [ ]:
# Save happy faces (Yahoo Images)
outDir = '/Users/abock/Desktop/HappyFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
    
happyPerson = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXdqJYFYvE0A0juLuLkF?p=happy+person&ei=UTF-8&iscqry=&fr=sfp'
url = happyPerson
saveURLimages(url,outDir)

happyHumanFace = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXyAJoFYWHMApB6JzbkF;_ylu=X3oDMTBsZ29xY3ZzBHNlYwNzZWFyY2gEc2xrA2J1dHRvbg--;_ylc=X1MDOTYwNjI4NTcEX3IDMgRhY3RuA2NsawRiY2sDNmltaTE4bGM2N29ybyUyNmIlM0QzJTI2cyUzRDI5BGNzcmNwdmlkAzNzeFdhVEl3Tmk1cFdrRkZXR1BqZUFkSE5qVXVPQUFBQUFDdHdqNncEZnIDc2ZwBGZyMgNzYS1ncARncHJpZANLQm5lbEt4X1JOeUlzNXY5WC40cVRBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzE2BHF1ZXJ5A2hhcHB5IGh1bWFuIGZhY2UEdF9zdG1wAzE0ODQ4NTkwNzYEdnRlc3RpZANudWxs?gprid=KBnelKx_RNyIs5v9X.4qTA&pvid=3sxWaTIwNi5pWkFFWGPjeAdHNjUuOAAAAACtwj6w&p=happy+human+face&fr=sfp&fr2=sb-top-images.search.yahoo.com&ei=UTF-8&n=60&x=wrt'
url = happyHumanFace
saveURLimages(url,outDir)



### Sad Faces

In [ ]:
# Output directory
outDir = '/Users/abock/Desktop/SadFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
   
# Yahoo Images
sadPerson = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXZ6JYFYl68AvSGJzbkF;_ylu=X3oDMTBsZ29xY3ZzBHNlYwNzZWFyY2gEc2xrA2J1dHRvbg--;_ylc=X1MDOTYwNjI4NTcEX3IDMgRhY3RuA2NsawRiY2sDNmltaTE4bGM2N29ybyUyNmIlM0QzJTI2cyUzRDI5BGNzcmNwdmlkA3VBdHlfekl3Tmk1cFdrRkZXR1BqZUFDaE5qVXVPQUFBQUFDZUhiR3oEZnIDc2ZwBGZyMgNzYS1ncARncHJpZANnb3NOVG13V1RTdVc1MXA5ZmhMZGlBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzEwBHF1ZXJ5A3NhZCBwZXJzb24EdF9zdG1wAzE0ODQ4NTg3NjAEdnRlc3RpZANudWxs?gprid=gosNTmwWTSuW51p9fhLdiA&pvid=uAty_zIwNi5pWkFFWGPjeAChNjUuOAAAAACeHbGz&p=sad+person&fr=sfp&fr2=sb-top-images.search.yahoo.com&ei=UTF-8&n=60&x=wrt'
url = sadPerson
saveURLimages(url,outDir)

sadHumanFace = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXmHJYFY_lwACdOJzbkF;_ylu=X3oDMTBsZ29xY3ZzBHNlYwNzZWFyY2gEc2xrA2J1dHRvbg--;_ylc=X1MDOTYwNjI4NTcEX3IDMgRhY3RuA2NsawRiY2sDNmltaTE4bGM2N29ybyUyNmIlM0QzJTI2cyUzRDI5BGNzcmNwdmlkA2N3Q2RJREl3Tmk1cFdrRkZXR1BqZUFUVU5qVXVPQUFBQUFDZTZzVXEEZnIDc2ZwBGZyMgNzYS1ncARncHJpZANKZHp4ZFBPMVFfSzFlYlZTTkpycGhBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzE0BHF1ZXJ5A3NhZCBodW1hbiBmYWNlBHRfc3RtcAMxNDg0ODU5MDA5BHZ0ZXN0aWQDbnVsbA--?gprid=JdzxdPO1Q_K1ebVSNJrphA&pvid=cwCdIDIwNi5pWkFFWGPjeATUNjUuOAAAAACe6sUq&p=sad+human+face&fr=sfp&fr2=sb-top-images.search.yahoo.com&ei=UTF-8&n=60&x=wrt'
url = sadHumanFace
saveURLimages(url,outDir)

In [ ]:
# # Set paths, variable values
# dbDir = '/Users/abock/Desktop/InsightProject/faceDatabases/FERG/'

# print("Loading input images...")
# t0 = time()

# # Find the images and emotion labels
# for subDir in glob.glob(dbDir + '*/'):
#     subject = os.path.basename(os.path.dirname(subDir))
    
#     for faceDir in glob.glob(subDir + '*/'):
#         tmp = os.path.basename(os.path.dirname(faceDir))
#         thisEmotion = tmp[len(subject)+1:]
#         for imFile in glob.glob(faceDir + '*.png'):
#             # Load the image
#             rawImg = cv2.imread(imFile)

#             # Detect faces in the image
#             faces = faceCascade.detectMultiScale(
#             rawImg,
#             scaleFactor=scaleSize,
#             minNeighbors=numNeighbors,
#             minSize=(faceSize, faceSize),
#             flags = cv2.CASCADE_SCALE_IMAGE
#             )

#             # If a face is detected
#             if len(faces):
#                 # Convert to gray
#                 gImg = cv2.cvtColor(rawImg, cv2.COLOR_BGR2GRAY)

#                 # Resize just the face
#                 x,y,w,h = faces[0,:]
#                 faceImg = gImg[y:y+h,x:x+w]
#                 img = cv2.resize(faceImg,(outImgSize,outImgSize),interpolation = cv2.INTER_CUBIC)

# #                 cv2.imshow('Image',img)
# #                 cv2.waitKey(30)

#                 # Flatten
#                 l = img.shape[0] * img.shape[1]
#                 img = img.reshape(1, l)
                
#                 # Append to subjectEmotions
#                 subjectNames.append(subject)
#                 # Append to subjectEmotions
#                 subjectEmotions.append(thisEmotion)
#                 # Append to subjectFiles
#                 subjectFiles.append(imFile) 
#                 # Append to output matrices
#                 subjectImages.append(img[0])
                
# print("done in %0.3fs" % (time() - t0))